# Luftqualitätsdaten: Import, Transformation & Excel-Export

Dieses Notebook führt folgende Schritte aus:
1. **Extraktion** von CSV-Daten aus verschachtelten ZIP-Archiven.
2. **Bereinigung** und Kombination der Zeitstempel.
3. **Pivotierung** in ein Analyse-freundliches Format (Wide-Format).
4. **Export** der Ergebnisse in eine Excel-Datei.

In [1]:
import pandas as pd
import os
import zipfile

data_dir = 'raw_data'
li = []

if not os.path.exists(data_dir):
    print(f"Ordner '{data_dir}' nicht gefunden. Bitte erstellen.")
else:
    for zip_filename in os.listdir(data_dir):
        if zip_filename.endswith(".zip"):
            zip_path = os.path.join(data_dir, zip_filename)
            with zipfile.ZipFile(zip_path, "r") as z:
                for member_path in z.namelist():
                    if member_path.endswith(".csv") and "__MACOSX" not in member_path:
                        try:
                            with z.open(member_path) as f:
                                df_temp = pd.read_csv(
                                    f, 
                                    sep=';', 
                                    decimal=',', 
                                    encoding='utf-8'
                                )
                                li.append(df_temp)
                        except Exception as e:
                            print(f"Fehler bei {member_path}: {e}")

    if li:
        df = pd.concat(li, axis=0, ignore_index=True).drop_duplicates()
        print(f"Erfolgreich geladen: {len(df)} Zeilen.")
    else:
        print("Keine Daten gefunden.")

Erfolgreich geladen: 15423143 Zeilen.


## Daten transformieren

In [2]:
if 'df' in locals() and not df.empty:
    # Zeitstempel konvertieren
    df['Timestamp'] = pd.to_datetime(df['Datum'] + ' ' + df['Uhrzeit'], dayfirst=True, errors='coerce')
    
    # Pivotieren (Schadstoffe in Spalten)
    index_cols = ['Stationscode', 'Stationsname', 'Stationsumgebung', 'Art der Station', 'Timestamp', 'Einheit']
    df_wide = df.pivot_table(
        index=index_cols, 
        columns='Schadstoff', 
        values='Messwert',
        aggfunc='first'
    ).reset_index()
    
    df_wide.columns.name = None
    display(df_wide.head())

,Stationscode,Stationsname,Stationsumgebung,Art der Station,Timestamp,Einheit,Feinstaub (PM₁₀),"Feinstaub (PM₂,₅)",Ozon (O₃),Schwefeldioxid (SO₂),Stickstoffdioxid (NO₂)
0,DEBB021,Potsdam-Zentrum,städtisches Gebiet,Hintergrund,2023-01-01 01:00:00,µg/m³,-,-,54,4,8
1,DEBB021,Potsdam-Zentrum,städtisches Gebiet,Hintergrund,2023-01-01 02:00:00,µg/m³,-,-,59,2,4
2,DEBB021,Potsdam-Zentrum,städtisches Gebiet,Hintergrund,2023-01-01 03:00:00,µg/m³,-,-,59,1,4
3,DEBB021,Potsdam-Zentrum,städtisches Gebiet,Hintergrund,2023-01-01 04:00:00,µg/m³,-,-,60,1,4
4,DEBB021,Potsdam-Zentrum,städtisches Gebiet,Hintergrund,2023-01-01 05:00:00,µg/m³,-,-,61,"0,8",4


## Export nach Excel
Wir speichern die bereinigten Daten nun als `.xlsx` Datei ab.

In [3]:
#output_file = "Luftqualitaet_Zusammenfassung.xlsx"
output_file = "Luftqualitaet_Zusammenfassung.csv"

try:
    # index=False verhindert, dass die Zeilennummern von Pandas mitgespeichert werden
    # df_wide.to_excel(output_file, index=False, engine='openpyxl')
    df_wide.to_csv(output_file, sep=';', index=False, encoding='utf-8-sig')
    print(f"✅ Datei erfolgreich gespeichert: {output_file}")
except Exception as e:
    print(f"❌ Fehler beim Speichern: {e}")

✅ Datei erfolgreich gespeichert: Luftqualitaet_Zusammenfassung.csv
